In [1]:
# Atividade Avaliativa de Lógica Computacional
# Geração da tabela-verdade para demonstração da propriedade do Dilema Construtivo (DC)
# ((p -> q) AND (r -> s) AND (p OR r)) -> (q OR s)

%pip install pandas
%pip install tabulate

import pandas as pd
from tabulate import tabulate

def conversao_booleano(valor):
    """
    Converte o valor fornecido para um booleano.
    
    Aceita valores como 0, 1, 'V', 'F', True, False e suas representações em string.
    
    Args:
        valor: O valor a ser convertido para booleano.
        
    Returns:
        bool: O valor convertido.
    
    Raises:
        ValueError: Se o valor fornecido não puder ser convertido.
    """
    valores_map = {
        0: False, 1: True,
        'V': True, 'F': False,
        'True': True, 'False': False,
        True: True, False: False
    }
    
    if valor in valores_map:
        return valores_map[valor]
    elif isinstance(valor, bool):
        return valor
    else:
        raise ValueError("Valor inválido para conversão. Use 0, 1, 'V', 'F', True, ou False.")

def conjuncao(x, y):
    """
    Calcula a conjunção (AND) lógica entre dois valores.
    
    Args:
        x: O primeiro valor a ser convertido para booleano.
        y: O segundo valor a ser convertido para booleano.
        
    Returns:
        str: 'V' se ambos os valores forem verdadeiros, caso contrário, 'F'.
    """
    return 'V' if conversao_booleano(x) and conversao_booleano(y) else 'F'

def disjuncao(x, y):
    """
    Calcula a disjunção (OR) lógica entre dois valores.
    
    Args:
        x: O primeiro valor a ser convertido para booleano.
        y: O segundo valor a ser convertido para booleano.
        
    Returns:
        str: 'V' se pelo menos um dos valores for verdadeiro, caso contrário, 'F'.
    """
    return 'V' if conversao_booleano(x) or conversao_booleano(y) else 'F'

def condicional(x, y):
    """
    Calcula a condicional (implicação) lógica entre dois valores.
    
    A implicação lógica 'x -> y' é verdadeira exceto quando x é verdadeiro e y é falso.
    
    Args:
        x: O primeiro valor a ser convertido para booleano (antecedente).
        y: O segundo valor a ser convertido para booleano (consequente).
        
    Returns:
        str: 'V' se a implicação lógica for verdadeira, caso contrário, 'F'.
    """
    return 'V' if not conversao_booleano(x) or conversao_booleano(y) else 'F'

def bicondicional(x, y):
    """
    Calcula o bicondicional (equivalência lógica) entre dois valores.
    
    A equivalência lógica 'x <-> y' é verdadeira se ambos os valores forem iguais 
    (ou ambos verdadeiros ou ambos falsos).
    
    Args:
        x: O primeiro valor a ser convertido para booleano.
        y: O segundo valor a ser convertido para booleano.
        
    Returns:
        str: 'V' se a equivalência lógica for verdadeira, caso contrário, 'F'.
    """
    return 'V' if conversao_booleano(x) == conversao_booleano(y) else 'F'

def imprimir_tabela_verdade(df):
    """
    Imprime a tabela verdade em formato tabulado.
    
    Args:
        df (pd.DataFrame): DataFrame contendo a tabela verdade.
    """
    print(tabulate(df.values.tolist(), headers=list(df.columns), tablefmt='fancy_grid', showindex=False, colalign=("center",)*len(df.columns)))

def gerar_dataframe_base_tabela_verdade(proposicoes: list[str]) -> pd.DataFrame:
    """
    Gera um DataFrame base para a tabela verdade com base nas proposições fornecidas.
    
    Args:
        proposicoes (list[str]): Lista de proposições para a tabela verdade.
        
    Returns:
        pd.DataFrame: DataFrame contendo a tabela verdade base.
    """
    quantidade_proposicoes = len(proposicoes)
    numero_linhas_tabela_verdade = 2 ** quantidade_proposicoes
    
    tabela = []
    for i in range(quantidade_proposicoes):
        bloco_tamanho = 2 ** (quantidade_proposicoes - i - 1)
        coluna = []
        while len(coluna) < numero_linhas_tabela_verdade:
            coluna.extend(['V'] * bloco_tamanho)
            coluna.extend(['F'] * bloco_tamanho)
        tabela.append(coluna[:numero_linhas_tabela_verdade])
    
    df_base = pd.DataFrame(tabela)
    
    return df_base

def avaliar_formula(coluna, passo):
    """
    Avalia uma fórmula lógica utilizando os operadores lógicos AND, OR, implicação e equivalência.
    
    Args:
        coluna (pd.Series): Coluna do DataFrame contendo os valores booleanos.
        passo (str): Identificador do passo de resolução que contém a fórmula a ser avaliada.
        
    Returns:
        str: Resultado da operação lógica.
        
    Raises:
        ValueError: Se a fórmula não tiver exatamente três partes ou se o operador for desconhecido.
    """
    valor_passo = passos_resolucao[passo]
    elementos_formula = valor_passo.split()

    if len(elementos_formula) != 3:
        raise ValueError("A fórmula sem parênteses deve ter exatamente três partes: proposição1, operador, proposição2.")
    
    proposicao1 = coluna[elementos_formula[0]]
    operador = elementos_formula[1]
    proposicao2 = coluna[elementos_formula[2]]

    if operador == 'AND':
        return conjuncao(proposicao1, proposicao2)
    elif operador == 'OR':
        return disjuncao(proposicao1, proposicao2)
    elif operador == '->':
        return condicional(proposicao1, proposicao2)
    elif operador == '<->':
        return bicondicional(proposicao1, proposicao2)
    else:
        raise ValueError(f"Operador desconhecido: {operador}")

def resolucao_tabela_verdade(formula: str, passos_resolucao: dict[str, str]) -> pd.DataFrame:
    """
    Resolve uma fórmula lógica e retorna a tabela verdade correspondente.
    
    Args:
        formula (str): A fórmula lógica a ser resolvida.
        passos_resolucao (dict[str, str]): Dicionário contendo os passos de resolução e suas fórmulas correspondentes.
        
    Returns:
        pd.DataFrame: DataFrame contendo a tabela verdade para a fórmula fornecida.
    """
    proposicoes = sorted(set(filter(str.isalpha, formula.replace('AND', '').replace('OR', '').replace('->', '').replace('(', '').replace(')', ''))))
    
    df_base = gerar_dataframe_base_tabela_verdade(proposicoes)

    df_base = df_base.T
    df_base.columns = proposicoes

    for passo in passos_resolucao:
        for index, coluna in df_base.iterrows():
            df_base.at[index, passo] = avaliar_formula(coluna, passo[0])

    return df_base

# Passo-a-passo utilizado para resolução da equação do dilema construtivo
# reduzindo sempre a análise a 2 proposições para avaliação

# a => p -> q
# b => r -> s
# c => p OR r
# d => (p -> q) AND (r -> s) => a AND b
# e => (p -> q) AND (r -> s) AND (p OR r) => d AND c
# f => q OR s
# g => ((p -> q) AND (r -> s) AND (p OR r)) -> (q OR s) => e -> f

passos_resolucao = {
        'a': 'p -> q',
        'b': 'r -> s',
        'c': 'p OR r',
        'd': 'a AND b',
        'e': 'd AND c',
        'f': 'q OR s',
        'g': 'e -> f',
    }

formula_dilema_construtivo = "((p -> q) AND (r -> s) AND (p OR r)) -> (q OR s)"
dataFrame = resolucao_tabela_verdade(formula_dilema_construtivo, passos_resolucao)

dataFrame.rename(columns={'a': 'p -> q'}, inplace=True)
dataFrame.rename(columns={'b': 'r -> s'}, inplace=True)
dataFrame.rename(columns={'c': 'p OR r'}, inplace=True)
dataFrame.rename(columns={'d': '(p -> q) AND (r -> s)'}, inplace=True)
dataFrame.rename(columns={'e': '(p -> q) AND (r -> s) AND (p OR r)'}, inplace=True)
dataFrame.rename(columns={'f': 'q OR s'}, inplace=True)
dataFrame.rename(columns={'g': '((p -> q) AND (r -> s) AND (p OR r)) -> (q OR s)'}, inplace=True)

imprimir_tabela_verdade(dataFrame)

#TrabalhoFinalizado

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
╒═════╤═════╤═════╤═════╤══════════╤══════════╤══════════╤═════════════════════════╤══════════════════════════════════════╤══════════╤════════════════════════════════════════════════════╕
│  p  │  q  │  r  │  s  │  p -> q  │  r -> s  │  p OR r  │  (p -> q) AND (r -> s)  │  (p -> q) AND (r -> s) AND (p OR r)  │  q OR s  │  ((p -> q) AND (r -> s) AND (p OR r)) -> (q OR s)  │
╞═════╪═════╪═════╪═════╪══════════╪══════════╪══════════╪═════════════════════════╪══════════════════════════════════════╪══════════╪════════════════════════════════════════════════════╡
│  V  │  V  │  V  │  V  │    V     │    V     │    V     │            V            │                  V                   │    V     │                         V                          │
├─────┼─────┼─────┼─────┼──────────┼──────────┼──────────┼─────────────────────────┼────────────────────────────────